In [8]:
import pandas as pd
import numpy as np
import os
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [9]:
path_cnad = "/nethome/rtandon32/ebm/ebm_experiments/experiment_scripts/real_data/TADPOLE_all_aligned/df_all_first_cnad_no_5ventricle.csv"
path_mci = "/nethome/rtandon32/ebm/ebm_experiments/experiment_scripts/real_data/TADPOLE_all_aligned/df_all_first_mci_no_5_ventricle.csv"
long_path = "/nethome/rtandon32/ebm/ebm_experiments/experiment_scripts/real_data/TADPOLE_all_aligned/df_all_long_no_5_ventricle.csv"

In [10]:
df_cnad = pd.read_csv(path_cnad)
df_mci = pd.read_csv(path_mci)
df_long = pd.read_csv(long_path)
df_cnad["EXAMDATE"] = pd.to_datetime(df_cnad["EXAMDATE"])
df_mci["EXAMDATE"] = pd.to_datetime(df_mci["EXAMDATE"])
df_long["EXAMDATE"] = pd.to_datetime(df_long["EXAMDATE"])

In [15]:
df_cnad.shape

(850, 130)

In [16]:
df_mci.shape

(808, 130)

In [17]:
df_cnad["Gender"] = df_cnad["PTGENDER"].map({"Male":1, "Female":0})
df_mci["Gender"] = df_mci["PTGENDER"].map({"Male":1, "Female":0})
df_long["Gender"] = df_long["PTGENDER"].map({"Male":1, "Female":0})
df_cnad["APOE4"].fillna(1, inplace=True)
df_mci["APOE4"].fillna(1, inplace=True)
df_long["APOE4"].fillna(1, inplace=True)

In [18]:
df_long

,3rd-Ventricle,4th-Ventricle,Brain-Stem,CC_Anterior,CC_Central,CC_Mid_Anterior,CC_Mid_Posterior,CC_Posterior,CSF,CortexVol,...,DX_bl,EXAMDATE,PTEDUCAT,PTGENDER,PTID,VISCODE,align_err_days,date,sid,Gender
0,2016,1594,21966,718,330,334,341,747,1744,428237.8199,...,CN,2006-05-09,18,Male,002_S_0295,bl,21,4/18/06,002_S_0295,1
1,1923,1571,21688,715,324,330,327,706,1787,431384.6187,...,CN,2006-11-02,18,Male,002_S_0295,m06,0,11/2/06,002_S_0295,1
2,2274,1717,20363,743,352,361,309,844,1537,454812.9567,...,CN,2007-06-01,16,Female,002_S_0413,m12,0,6/1/07,002_S_0413,0
3,2624,1744,19964,715,350,341,296,843,1543,432037.1380,...,CN,2008-07-31,16,Female,002_S_0413,m24,0,7/31/08,002_S_0413,0
4,2341,1596,20078,719,362,342,304,795,1539,446604.6537,...,CN,2009-04-30,16,Female,002_S_0413,m36,4,5/4/09,002_S_0413,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5544,3337,3008,20672,460,276,199,195,751,1948,367043.4775,...,EMCI,2012-08-20,20,Female,941_S_4764,m03,0,8/20/12,941_S_4764,0
5545,3328,3418,20491,459,247,205,204,741,2233,378188.3252,...,EMCI,2013-06-10,20,Female,941_S_4764,m12,0,6/10/13,941_S_4764,0
5546,3499,3330,20179,413,242,188,178,739,2135,352155.9865,...,EMCI,2014-06-16,20,Female,941_S_4764,m24,0,6/16/14,941_S_4764,0
5547,1267,1305,20275,922,428,395,331,947,1156,391568.0745,...,SMC,2013-06-11,16,Female,941_S_5193,bl,13,5/29/13,941_S_5193,0


In [19]:
all(df_cnad.columns == df_mci.columns)

True

In [20]:
all(df_cnad.columns == df_long.columns)

True

In [21]:
fld = ["Gender", "AGE", "PTEDUCAT", "APOE4"]
cols = df_cnad.columns[:118].tolist()
model_list = []
df_cnad_resid = pd.DataFrame(columns = cols)
for i, c in enumerate(cols):
    model = sm.OLS(df_cnad[c], sm.add_constant(df_cnad[fld]))
    results = model.fit()
    model_list.append(results)
    df_cnad_resid[c] = results.resid
    
    

In [22]:
df_mci_resid = pd.DataFrame(columns = cols)
for i, c in enumerate(cols):
    df_mci_resid[c] = df_mci[c] - model_list[i].predict(sm.add_constant(df_mci[fld]))

In [24]:
df_long_resid = pd.DataFrame(columns = cols)
for i, c in enumerate(cols):
    df_long_resid[c] = df_long[c] - model_list[i].predict(sm.add_constant(df_long[fld]))

In [25]:
df_cnad_resid_final = pd.concat([df_cnad_resid, df_cnad.iloc[:,118:]], axis=1)
df_mci_resid_final = pd.concat([df_mci_resid, df_mci.iloc[:,118:]], axis=1)
df_long_resid_final = pd.concat([df_long_resid, df_long.iloc[:,118:]], axis=1)

In [26]:
df_long_resid_final

,3rd-Ventricle,4th-Ventricle,Brain-Stem,CC_Anterior,CC_Central,CC_Mid_Anterior,CC_Mid_Posterior,CC_Posterior,CSF,CortexVol,...,DX_bl,EXAMDATE,PTEDUCAT,PTGENDER,PTID,VISCODE,align_err_days,date,sid,Gender
0,-466.244891,-757.156844,655.746528,29.748840,5.656936,15.926647,44.492168,-104.026785,-44.067961,37761.248283,...,CN,2006-05-09,18,Male,002_S_0295,bl,21,4/18/06,002_S_0295,1
1,-559.244891,-780.156844,377.746528,26.748840,-0.343064,11.926647,30.492168,-145.026785,-1.067961,40908.047083,...,CN,2006-11-02,18,Male,002_S_0295,m06,0,11/2/06,002_S_0295,1
2,636.116064,-158.929618,742.354873,35.662452,12.410384,22.973607,-2.409478,-11.392500,114.047531,75991.970407,...,CN,2007-06-01,16,Female,002_S_0413,m12,0,6/1/07,002_S_0413,0
3,986.116064,-131.929618,343.354873,7.662452,10.410384,2.973607,-15.409478,-12.392500,120.047531,53216.151707,...,CN,2008-07-31,16,Female,002_S_0413,m24,0,7/31/08,002_S_0413,0
4,703.116064,-279.929618,457.354873,11.662452,22.410384,3.973607,-7.409478,-60.392500,116.047531,67783.667407,...,CN,2009-04-30,16,Female,002_S_0413,m36,4,5/4/09,002_S_0413,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5544,1514.551114,997.362421,1133.820612,-206.495419,-50.354346,-124.085538,-103.367136,-85.400551,407.603358,-8129.073221,...,EMCI,2012-08-20,20,Female,941_S_4764,m03,0,8/20/12,941_S_4764,0
5545,1505.551114,1407.362421,952.820612,-207.495419,-79.354346,-118.085538,-94.367136,-95.400551,692.603358,3015.774479,...,EMCI,2013-06-10,20,Female,941_S_4764,m12,0,6/10/13,941_S_4764,0
5546,1676.551114,1319.362421,640.820612,-253.495419,-84.354346,-135.085538,-120.367136,-97.400551,594.603358,-23016.564221,...,EMCI,2014-06-16,20,Female,941_S_4764,m24,0,6/16/14,941_S_4764,0
5547,-250.695711,-530.111576,475.427535,187.465908,77.970915,43.211247,8.563567,73.245354,-216.452212,5980.471884,...,SMC,2013-06-11,16,Female,941_S_5193,bl,13,5/29/13,941_S_5193,0


In [27]:
df_cnad_resid_final.to_csv("/nethome/rtandon32/ebm/ebm_experiments/experiment_scripts/real_data/TADPOLE_all_aligned/df_all_first_cnad_no_5ventricle_adj.csv", index=False)
df_mci_resid_final.to_csv("/nethome/rtandon32/ebm/ebm_experiments/experiment_scripts/real_data/TADPOLE_all_aligned/df_all_first_mci_no_5ventricle_adj.csv", index=False)
df_long_resid_final.to_csv("/nethome/rtandon32/ebm/ebm_experiments/experiment_scripts/real_data/TADPOLE_all_aligned/df_all_long_no_5_ventricle_adj.csv", index=False)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.hist(df_mci_resid[cols[34]])
plt.hist(df_cnad_resid[cols[31]])


In [ ]:
df_cnad_resid.shape

In [ ]:
model_list[1].predict(sm.add_constant(df_mci[fld]))

In [ ]:
df_cnad_resid

In [ ]:
model = sm.OLS(df_cnad.iloc[:,:118], sm.add_constant(df_cnad[fld]))
results = model.fit()

In [ ]:
results.resid

In [ ]:
model.predict(sm.add_constant(df_mci[fld]))

In [ ]:
model.predict(df_)

In [ ]:


len(model_list)
# for m in model_list:
#     print(m.rsquared_adj)

In [ ]:
m.rsquared_adj

In [ ]:
results.resid

In [ ]:
df_mci[df_mci["APOE4"].isna()]["APOE4"]

In [ ]:
df_cnad.iloc[:,:119]